In [ ]:
%pip  uninstall chroma-hnswlib chromadb -y

In [ ]:
 %pip show hnswlib

In [ ]:
%pip install -U sentence-transformers  html2text lxml nest_asyncio chromadb==0.4.3
%pip install --upgrade "langchain[docarray]"  langchain-experimental langchain pydantic==1.10.12

In [ ]:
%reload_ext dotenv
%dotenv /notebooks/chainlang/my.env

In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import openai
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler



In [ ]:

llm = ChatOpenAI(model_name="gpt-4") 


INIT MODEL

In [ ]:
n_gpu_layers = 1  # Metal set to 1 is enough.
n_batch = 512  # Should be between 1 and n_ctx, consider the amount of RAM of your Apple Silicon Chip.
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="/notebooks/llama-2-13b-chat.Q6_K.gguf",
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=False,
    n_ctx=2048,
    n_gpu_layers=43
)

In [ ]:
memory = ConversationSummaryMemory(llm=llm,memory_key="chat_history",return_messages=True)

LOAD DOCUMENTS

In [ ]:

from langchain.document_loaders.sitemap import SitemapLoader
from bs4 import BeautifulSoup as Soup
from langchain.document_loaders.recursive_url_loader import RecursiveUrlLoader

# url = ""
# loader = SitemapLoader(web_path=url, extractor=lambda x: Soup(x, "html.parser").text)
url = ""
loader = RecursiveUrlLoader(
    url=url, max_depth=2, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()
print(len(docs))

from langchain.document_transformers import Html2TextTransformer

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(docs_transformed)

from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores.utils import filter_complex_metadata

db = Chroma.from_documents(
    filter_complex_metadata(all_splits),
    HuggingFaceEmbeddings(
        model_name="intfloat/multilingual-e5-large"
    ),
    persist_directory="./chroma_db2",
)

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import GPT4AllEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.utils import filter_complex_metadata

from langchain.embeddings import HuggingFaceEmbeddings

db = Chroma(
    persist_directory="./chroma_db2",
    embedding_function= HuggingFaceEmbeddings(
        model_name="intfloat/multilingual-e5-large"
    ),
)

In [ ]:

retriever = db.as_retriever(
    search_type="mmr", # Also test "similarity"
    search_kwargs={"k": 8},
)

In [ ]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool = create_retriever_tool(
    retriever, 
    "get_info_ppw",
    "Get information compagny."
)
tools = [tool]

In [ ]:

from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent


agent = create_conversational_retrieval_agent(
   llm,tools, verbose=True,handle_parsing_errors=True
)

EXECUTE QUERY

In [ ]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
from langchain.chains import RetrievalQA 
from langchain.chains import ConversationalRetrievalChain

#qa_chain = RetrievalQA.from_chain_type(   
 # llm=llm,   
 # chain_type="stuff",   
  #retriever=retriever,   
 # chain_type_kwargs={"prompt": QA_CHAIN_PROMPT} 
#) 

qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever=retriever, memory=memory)

In [ ]:
result = qa_chain ("how many dev is there at google") 
print(result["answer"])

CHAT

In [ ]:
import os
os.environ["DISABLE_NEST_ASYNCIO"]="True"

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response =  agent({ "input" : message }) 
    return gpt_response["output"]

gr.ChatInterface(predict).launch(share=True)